In [1]:
import sys
import spikeinterface as si
import matplotlib.pyplot as plt
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import probeinterface

from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe, plot_probegroup
from probeinterface import generate_dummy_probe, generate_linear_probe
from probeinterface import write_probeinterface, read_probeinterface
from probeinterface import write_prb, read_prb
import os

global_job_kwargs = dict(n_jobs=6, chunk_duration="1s")
si.set_global_job_kwargs(**global_job_kwargs)

/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = os.listdir("/media/ubuntu/sda/data/mouse6/ns4/natural_image")

In [3]:
recording_raw = {}
for i in range(len(data_dir)):
    recording_raw[i] = se.read_blackrock(file_path=f'/media/ubuntu/sda/data/mouse6/ns4/natural_image/{data_dir[i]}')

In [7]:
recording_grating = {}
grating_dir = os.listdir("/media/ubuntu/sda/data/mouse6/ns4/grating")
for i in range(len(grating_dir)):
    recording_grating[i] = se.read_blackrock(file_path=f'/media/ubuntu/sda/data/mouse6/ns4/grating/{grating_dir[i]}')

In [9]:
recording_concatenated = si.concatenate_recordings([recording_raw[0], recording_raw[1], recording_raw[2], recording_raw[3],
                                            recording_raw[4], recording_raw[5], recording_raw[6], recording_raw[7],
                                            recording_raw[8], recording_raw[9], recording_raw[10], recording_raw[11],
                                            recording_raw[12], recording_raw[13], recording_raw[14], recording_raw[15],
                                            recording_grating[0], recording_grating[1], recording_grating[2], recording_grating[3],
                                            recording_grating[4], recording_grating[5], recording_grating[6], recording_grating[7],
                                            recording_grating[8], recording_grating[9], recording_grating[10], recording_grating[11],
                                            recording_grating[12], recording_grating[13]])

In [10]:
output_folder = '/media/ubuntu/sda/data/mouse6/output/concat/grating_natural_image'

In [ ]:
recording_recorded = recording_concatenated.remove_channels(["98", '31', '32'])

probe_30channel = read_probeinterface("/media/ubuntu/sda/data/probe.json")
recording_recorded = recording_recorded.set_probegroup(probe_30channel)

recording_cmr = recording_recorded
recording_f = spre.bandpass_filter(recording_recorded, freq_min=300, freq_max=3000)
print(recording_f)
recording_cmr = spre.common_reference(recording_f, reference="global", operator="median")
print(recording_cmr)

# this computes and saves the recording after applying the preprocessing chain
recording_preprocessed = recording_cmr.save(format="binary")
print(recording_preprocessed)

sorting_kilosort4 = ss.run_sorter(sorter_name="kilosort4", recording=recording_preprocessed, output_folder=output_folder + "/kilosort4")
analyzer_kilosort4 = si.create_sorting_analyzer(sorting=sorting_kilosort4, recording=recording_preprocessed, format='binary_folder', folder=output_folder + '/analyzer_kilosort4_binary')

extensions_to_compute = [
    "random_spikes",
    "waveforms",
    "noise_levels",
    "templates",
    "spike_amplitudes",
    "unit_locations",
    "spike_locations",
    "correlograms",
    "template_similarity"
]

extension_params = {
    "unit_locations": {"method": "center_of_mass"},
    "spike_locations": {"ms_before": 0.1},
    "correlograms": {"bin_ms": 0.1},
    "template_similarity": {"method": "cosine_similarity"}
}

analyzer_kilosort4.compute(extensions_to_compute, extension_params=extension_params)

qm_params = sqm.get_default_qm_params()
analyzer_kilosort4.compute("quality_metrics", qm_params)

        

BandpassFilterRecording: 30 channels - 10000.0Hz - 1 segments - 787,652,701 samples 
                         78,765.27s (21.88 hours) - int16 dtype - 44.01 GiB
CommonReferenceRecording: 30 channels - 10000.0Hz - 1 segments - 787,652,701 samples 
                          78,765.27s (21.88 hours) - int16 dtype - 44.01 GiB
Use cache_folder=/tmp/spikeinterface_cache/tmpp_wef4ve/078NLN8K
write_binary_recording 
n_jobs=6 - samples_per_chunk=10,000 - chunk_memory=585.94 KiB - total_memory=3.43 MiB - chunk_duration=1.00s


write_binary_recording: 100%|██████████| 78766/78766 [06:35<00:00, 199.35it/s]
/tmp/ipykernel_2392644/1191846257.py:16: DeprecationWarning: `output_folder` is deprecated and will be removed in version 0.103.0 Please use folder instead
  sorting_kilosort4 = ss.run_sorter(sorter_name="kilosort4", recording=recording_preprocessed, output_folder=output_folder + "/kilosort4")


BinaryFolderRecording: 30 channels - 10000.0Hz - 1 segments - 787,652,701 samples 
                       78,765.27s (21.88 hours) - int16 dtype - 44.01 GiB


 67%|██████▋   | 4/6 [10:11:07<3:58:15, 7147.74s/it] 